In [1]:
#DES Decryption 
'''input 64bits = 8 bytes block'''

import nbimporter,os
from subkey_generation import sub_key
from mangler_function import mangler_func

class des_decrypt():
    
    #initialisation
    def __init__(self,cipher_16hex,key_16hex):
        #plaintext 64bits is string (in hex form) 
        self.cipher_16hex = cipher_16hex
        self.key_16hex = key_16hex
    
        self.ip = [58,50,42,34,26,18,10,2,
                   60,52,44,36,28,20,12,4,
                   62,54,46,38,30,22,14,6,
                   64,56,48,40,32,24,16,8,
                   57,49,41,33,25,17,9,1,
                   59,51,43,35,27,19,11,3,
                   61,53,45,37,29,21,13,5,
                   63,55,47,39,31,23,15,7]

        self.ipinverse = [40,8,48,16,56,24,64,32,
                          39,7,47,15,55,23,63,31,
                          38,6,46,14,54,22,62,30,
                          37,5,45,13,53,21,61,29,
                          36,4,44,12,52,20,60,28,
                          35,3,43,11,51,19,59,27,
                          34,2,42,10,50,18,58,26,
                          33,1,41,9,49,17,57,25]
        
    def ini_permu(self,bit): 
        ip_64bit = ''
        for index in self.ip:
            ip_64bit = ip_64bit + bit[index-1]
        return ip_64bit   
    
    def iniver_permu(self,bit):
        ipinv_64bit = '' 
        for index in self.ipinverse:
            ipinv_64bit = ipinv_64bit + bit[index-1]
        return ipinv_64bit

    def split_in_half32(self,bit):
        L,R = bit[:32], bit[32:]
        return L,R
    
    def XOR(self,bit1,bit2):
        xor_result = ''
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r)
        return xor_result    
    
    def bin_to_hex(self,bit):
        bin_to_decimal = int(bit,2)
        decimal_to_hex = hex(bin_to_decimal)[2:].zfill(16)
        return decimal_to_hex.upper()
    
    def hex_to_bin(self,bit):
        hex_to_decimal = int(bit,16)
        decimal_to_bin = bin(hex_to_decimal)[2:].zfill(64)
        return decimal_to_bin
    
    #-----------decryption-------------
    
    def generate(self):
        #transform hex to bin
        ini_64bit = self.hex_to_bin(self.cipher_16hex)
        key_64bit = self.hex_to_bin(self.key_16hex)
        
        #permutation
        ip64 = self.ini_permu(ini_64bit)
        L0,R0 = self.split_in_half32(ip64)
        
        '''key generator'''
        subkey = sub_key(key_64bit)
        roundkey_list = subkey.generate_subkey()
        
        '''reverse order of keys'''
        reverse_key = list()
        while roundkey_list:
            top = roundkey_list.pop()
            reverse_key.append(top)
        
        '''15 rounds'''
        for i in range(15):
            f = mangler_func(R0,reverse_key[i])
            temp,L0 = L0,R0
            R0 = self.XOR(temp,f.generate_f())
            
        #return self.bin_to_hex(L0+R0)        
        
        '''16 round (no swapping)'''
        f16 = mangler_func(R0,reverse_key[15])
        L16,R16 = self.XOR(L0,f16.generate_f()),R0
        
        #return self.bin_to_hex(L16+R16)
        
        '''final permutation'''
        final_bit = L16 + R16
        result = self.iniver_permu(final_bit)
        return self.bin_to_hex(result)
        

key_16hex = 'AABB09182736CCDD'
ciphertxt = 'C0B7A8D05F3A829C'

decrypt = des_decrypt(ciphertxt,key_16hex)
decrypt.generate()


Importing Jupyter notebook from subkey_generation.ipynb
Importing Jupyter notebook from mangler_function.ipynb


'123456ABCD132536'